In [4]:
import abc
import numpy as np
from collections import namedtuple
from scipy.stats import norm


# Inputs: 
#    F_t         = price of underlying
#    X_t         = strike
#    t           = time to expiration
#    v           = implied volatility
#    r           = risk free rate
#    q           = dividend payment
#    b           = cost of carry
# Outputs: 
#    value       = price of the option
#    delta       = 1st derivative of value w.r.t price of underlying
#    gamma       = 2nd derivative of value w.r.t price of underlying
#    theta       = 1st derivative of value w.r.t. time to expiration
#    vega        = 1st derivative of value w.r.t. implied volatility
#    rho         = 1st derivative of value w.r.t. risk free rates
Greeks = namedtuple("greeks", "value delta gamma theta vega rho")

In [5]:
# class GeneralisedEuropeanBlackScholes(abc.ABC):
    
#     def __init__(self, F_t, X_t, t, r, b, sigma):
#         self.F_t = F_t
#         self.X_t = X_t
#         self.t = t
#         self.r = r
#         self.b = b
#         self.sigma = sigma
        
#         # Pre-calculations
#         self.sqrt_t = np.sqrt(t)
#         self.d1 = (np.log(F_t/X_t) + (b + np.square(sigma)/2) * t) / (sigma * self.sqrt_t)
#         self.d2 = self.d1 - sigma * self.sqrt_t
        
#     def call(self):
#         # Unpack for readable equations
#         F_t = self.F_t
#         X_t = self.X_t
#         t = self.t
#         r = self.r
#         b = self.b
#         sigma = self.sigma
#         d1 = self.d1
#         d2 = self.d2
#         sqrt_t = self.sqrt_t
        
#         # Fair value price
#         value = F_t * np.exp((b - r)*t) * norm.cdf(d1) - X_t * np.exp(-r*t) * norm.cdf(d2)

#         # Greeks
#         delta = np.exp((b - r)*t) * norm.cdf(d1)
#         gamma = np.exp((b - r)*t) * norm.pdf(d1) / (F_t * sigma * sqrt_t)
#         theta = -(F_t * sigma * np.exp((b - r)*t) * norm.pdf(d1)) / (2*sqrt_t) - (b - r) * F_t * np.exp((b - r)*t) * norm.cdf(d1) - r * X_t * np.exp(-r * t) * norm.cdf(d2)
#         vega = np.exp((b - r)*t) * F_t * sqrt_t * norm.pdf(d1)
#         rho = X_t * t * np.exp(-r*t) * norm.cdf(d2)
        
#         return Greeks(value, delta, gamma, theta, vega, rho)
        
        
#     def put(self):
#         # Unpack for readable equations
#         F_t = self.F_t
#         X_t = self.X_t
#         t = self.t
#         r = self.r
#         b = self.b
#         sigma = self.sigma
#         d1 = self.d1
#         d2 = self.d2
#         sqrt_t = self.sqrt_t
        
#         # Fair value price
#         value = X_t * np.exp(-r*t) * norm.cdf(-d2) - (F_t * np.exp((b - r)*t) * norm.cdf(-d1))
        
#         # Greeks
#         delta = -np.exp((b - r)*t) * norm.cdf(-d1)
#         gamma = np.exp((b - r)*t) * norm.pdf(d1) / (F_t * sigma * sqrt_t)
#         theta = -(F_t * sigma * np.exp((b - r)*t) * norm.pdf(d1)) / (2*sqrt_t) + (b - r) * F_t * np.exp((b - r)*t) * norm.cdf(-d1) + r * X_t * np.exp(-r*t) * norm.cdf(-d2)
#         vega = np.exp((b - r)*t) * F_t * sqrt_t * norm.pdf(d1)
#         rho = -X_t * t * np.exp(-r*t) * norm.cdf(-d2)

#         return Greeks(value, delta, gamma, theta, vega, rho)

In [38]:
class GeneralisedEuropeanBlackScholes(abc.ABC):
    
    def __init__(self, name="GBS"):
        self.name = name
        
    def call(self, F_t, X_t, t, r, b, sigma):
        # Pre-calculations
        sqrt_t = np.sqrt(t)
        d1, d2 = self._distribution_parameters(F_t, X_t, t, r, b, sigma, sqrt_t)
        
        # Fair value price
        value = F_t * np.exp((b - r)*t) * norm.cdf(d1) - X_t * np.exp(-r*t) * norm.cdf(d2)

        # Greeks
        delta = np.exp((b - r)*t) * norm.cdf(d1)
        gamma = np.exp((b - r)*t) * norm.pdf(d1) / (F_t * sigma * sqrt_t)
        theta = -(F_t * sigma * np.exp((b - r)*t) * norm.pdf(d1)) / (2*sqrt_t) - (b - r) * F_t * np.exp((b - r)*t) * norm.cdf(d1) - r * X_t * np.exp(-r * t) * norm.cdf(d2)
        vega = np.exp((b - r)*t) * F_t * sqrt_t * norm.pdf(d1)
        rho = X_t * t * np.exp(-r*t) * norm.cdf(d2)
        
        return Greeks(value, delta, gamma, theta, vega, rho)
        
    def put(self, F_t, X_t, t, r, b, sigma):
        # Pre-calculations
        sqrt_t = np.sqrt(t)
        d1, d2 = self._distribution_parameters(F_t, X_t, t, r, b, sigma, sqrt_t)
        
        # Fair value price
        value = X_t * np.exp(-r*t) * norm.cdf(-d2) - (F_t * np.exp((b - r)*t) * norm.cdf(-d1))
        
        # Greeks
        delta = -np.exp((b - r)*t) * norm.cdf(-d1)
        gamma = np.exp((b - r)*t) * norm.pdf(d1) / (F_t * sigma * sqrt_t)
        theta = -(F_t * sigma * np.exp((b - r)*t) * norm.pdf(d1)) / (2*sqrt_t) + (b - r) * F_t * np.exp((b - r)*t) * norm.cdf(-d1) + r * X_t * np.exp(-r*t) * norm.cdf(-d2)
        vega = np.exp((b - r)*t) * F_t * sqrt_t * norm.pdf(d1)
        rho = -X_t * t * np.exp(-r*t) * norm.cdf(-d2)

        return Greeks(value, delta, gamma, theta, vega, rho)
    
    @staticmethod
    def _distribution_parameters(F_t, X_t, t, r, b, sigma, sqrt_t):
        # Static method allows for cleaner use of scipy.optimize.
        d1 = (np.log(F_t/X_t) + (b + np.square(sigma)/2) * t) / (sigma * sqrt_t)
        d2 = d1 - sigma * sqrt_t
        
        return d1, d2
    
    @property
    def _cost_of_carry(self, *args, **kwargs):
        raise NotImplementedError

In [43]:
class Black76Eur(GeneralisedEuropeanBlackScholes):
    
    def __init__(self):
        super().__init__(name="Black76")
        
    def call(self, F_t, X_t, t, r, sigma):
        b = self._cost_of_carry
        return super().call(F_t=F_t, X_t=X_t, t=t, r=r, b=b, sigma=sigma)
            
    def put(self, F_t, X_t, t, r, sigma):
        b = self._cost_of_carry
        return super().put(F_t=F_t, X_t=X_t, t=t, r=r, b=b, sigma=sigma)
    
    @property
    def cost_of_carry(self):
        # No cost of carry on the underlying (futures contract).
        return 0

In [44]:
F_t = 50
X_t = 55
t = 1/12
r = 0.04
sigma = 0.20


black76 = Black76Eur()
black76.call(F_t=F_t, X_t=X_t, t=t, r=r, sigma=sigma), black76.put(F_t=F_t, X_t=X_t, t=t, r=r, sigma=sigma)

(greeks(value=0.06213200829622734, delta=0.05223228374301364, gamma=0.03696551723344639, theta=-1.8457905813404705, vega=1.540229884726933, rho=0.2124568482378712),
 greeks(value=5.04549308856884, delta=-0.9444399323115097, gamma=0.03696551723344639, theta=-1.6464561381295657, vega=1.540229884726933, rho=-4.355624142012027))

In [ ]:
# Scroll down
def euro_implied_vol_76(option_type, fs, x, t, r, cp):
    b = 0
    return _gbs_implied_vol(option_type, fs, x, t, r, b, cp)

In [1]:
# scipy.optimize.newton

# _gbs_implied_vol wraps:
# _newton_implied_vol(_gbs, option_type, x, fs, t, b, r, cp, precision, max_steps)

# -> def _gbs()...

# _gbs is the equiv. of black76.call() and black76.put()

In [ ]:
def implied_volatility_black76_european(option_type, F_t, X_t, t, r, cp, precision, max_steps):
    black76 = Black76Eur()
    
    # black76.call(F_t=F_t, X_t=X_t, t=t, r=r, sigma=sigma)
    # black76.put(F_t=F_t, X_t=X_t, t=t, r=r, sigma=sigma)
    
    # Map options type to function to be optimised.
    pricing_funcs = {
        "put": black76.put,
        "call": black76.call,
    }
    
    b = black76.cost_of_carry
    
    return _implied_volatility_gbs(pricing_funcs["option_type"], F_t, X_t, t, r, b, cp, precision, max_steps)


def _implied_volatility_gbs(pricing_func, F_t, X_t, t, r, cp, precision, max_steps):
    ...
    

In [ ]:
# Calculate Implied Volatility with a Newton Raphson search
def _newton_implied_vol(val_fn, option_type, x, fs, t, b, r, cp, precision=.00001, max_steps=100):
    # make sure a valid option type was entered
    _test_option_type(option_type)

    # Estimate starting Vol, making sure it is allowable range
    v = _approx_implied_vol(option_type, fs, x, t, r, b, cp)
    v = max(_GBS_Limits.MIN_V, v)
    v = min(_GBS_Limits.MAX_V, v)

    # Calculate the value at the estimated vol
    value, delta, gamma, theta, vega, rho = val_fn(option_type, fs, x, t, r, b, v)
    min_diff = abs(cp - value)

    _debug("-----")
    _debug("Debug info for: _Newton_ImpliedVol()")
    _debug("    Vinitial={0}".format(v))

    # Newton-Raphson Search
    countr = 0
    while precision <= abs(cp - value) <= min_diff and countr < max_steps:

        v = v - (value - cp) / vega
        if (v > _GBS_Limits.MAX_V) or (v < _GBS_Limits.MIN_V):
            _debug("    Volatility out of bounds")
            break

        value, delta, gamma, theta, vega, rho = val_fn(option_type, fs, x, t, r, b, v)
        min_diff = min(abs(cp - value), min_diff)

        # keep track of how many loops
        countr += 1
        _debug("     IVOL STEP {0}. v={1}".format(countr, v))

    
    # check if function converged and return a value
    if abs(cp - value) < precision:
        # the search function converged
        return v
    else:
        # if the search function didn't converge, try a bisection search
        return _bisection_implied_vol(val_fn, option_type, fs, x, t, r, b, cp, precision, max_steps)